In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [ ]:
df_states = gpd.read_file('Data/us-states.json')
df_fires = gpd.read_file('Data/mtbs_fire_locations.geojson')

In [ ]:
df_states

In [ ]:
df_western_us = df_states[df_states['name'].isin(['California', 'Oregon', 'Washington', 'Nevada', 'Arizona', 'Idaho', 'Montana', 'Utah', 'Wyoming', 'Colorado', 'New Mexico'])]

In [ ]:
df_western_us.plot()

In [ ]:
gpd.GeoDataFrame(df_western_us).to_file('Data/western_us.geojson', driver='GeoJSON')

In [ ]:
df_fires["Ig_Date"] = pd.to_datetime(df_fires["Ig_Date"])

In [ ]:
def timestamp_to_year_part(df):
    date = df["Ig_Date"][0]
    year = int(date.strftime("%Y"))
    print('{} / {}'.format(date,year))
    df.year.values[:] = year
    return df

In [ ]:
df_fires

In [ ]:
date_mask_1985 = (df_fires["Ig_Date"] >= '1985-01-01') & (df_fires["Ig_Date"] <= '1985-12-31')
date_mask_2015 = (df_fires["Ig_Date"] >= '2015-01-01') & (df_fires["Ig_Date"] <= '2015-12-31')
fire_type_mask = (df_fires["Incid_Type"] != 'Prescribed Fire')
fire_size_mask = (df_fires["BurnBndAc"] >= 5000)
fire_location_mask = (df_fires["geometry"].within(df_western_us.unary_union))

In [ ]:
fires_1985 = df_fires[date_mask_1985 & fire_type_mask & fire_size_mask & fire_location_mask]
fires_2015 = df_fires[date_mask_2015 & fire_type_mask & fire_size_mask & fire_location_mask]

In [ ]:
fires_1985.to_csv('Data/fires_1985.csv')
fires_2015.to_csv('Data/fires_2015.csv')

In [ ]:
gpd.GeoDataFrame(fires_1985).to_file("Data/fires_1985.geojson", driver='GeoJSON')
gpd.GeoDataFrame(fires_2015).to_file("Data/fires_2015.geojson", driver='GeoJSON')

In [ ]:
df_fires.assign(year=np.int32(0))
df_fires = df_fires.apply(timestamp_to_year_part, axis=1)

In [ ]:
def fires_per_state(df):
    df["State"] = df["Event_ID"].apply(lambda x: x[0:2])
    return df

In [44]:
fires_2015 = fires_per_state(fires_2015)
fires_1985 = fires_per_state(fires_1985)

/home/jake/miniconda3/envs/rstools/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/jake/miniconda3/envs/rstools/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [46]:
burned_area_15 = pd.DataFrame(fires_2015.groupby("State")["BurnBndAc"].sum())
burned_area_85 = pd.DataFrame(fires_1985.groupby("State")["BurnBndAc"].sum())

In [47]:
states = {
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "ID": "Idaho",
    "MT": "Montana",
    "NV": "Nevada",
    "NM": "New Mexico",
    "OR": "Oregon",
    "UT": "Utah",
    "WA": "Washington",
    "WY": "Wyoming"
}

In [48]:
burned_area_15["State"] = burned_area_15.index.map(states)
burned_area_85["State"] = burned_area_85.index.map(states)

In [51]:
burned_area_15.sort_values(by="BurnBndAc", ascending=False, inplace=True)
burned_area_85.sort_values(by="BurnBndAc", ascending=False, inplace=True)

In [52]:
burned_area_15

,BurnBndAc,State
State,,
WA,1110427,Washington
CA,743419,California
ID,696881,Idaho
OR,559923,Oregon
MT,232693,Montana
AZ,90042,Arizona
NM,18848,New Mexico
WY,9845,Wyoming
NV,6508,Nevada


In [49]:
burned_area_15.to_csv('Data/burned_area_2015.csv')
burned_area_85.to_csv('Data/burned_area_1985.csv')

In [ ]:
ag = pd.read_csv("Data/us-ag-productivity-2004.csv")

In [ ]:
ag_out = ag[ag['state'].isin(['California', 'Oregon', 'Washington', 'Nevada', 'Arizona', 'Idaho', 'Montana', 'Utah', 'Wyoming', 'Colorado', 'New Mexico'])]

In [ ]:
ag_out.to_csv('Data/western_ag.csv')